In [1]:
import numpy as np
import pandas as pd
import sys
import os
from sklearn import preprocessing

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_columns', None)

In [3]:
path = r'C:\Users\U279014\Documents\H_Drive\7.AA Models\17.Supplier_Pricing_ReClassification_Optimization\data'
result_path = r'C:\Users\U279014\Documents\H_Drive\7.AA Models\17.Supplier_Pricing_ReClassification_Optimization\result'
file_full = r'CompetitveMatrixFullList.xlsx'
file_survey = r'Competitve Matrix.xlsx'
result_file = r'pricing_matrix_rank.csv'
result_financial_impact = r'finanical_impact.csv'

In [4]:
# standardize dataframe colume names
def col_name(df):
    """
    this is to trim the data_frame column names to a unique format:
    all case, replace space to underscore, remove parentheses
    param df:
        raw from share drive for
    return:
        polished data set with new column names
    """
    df.columns = df.columns.str.strip().str.lower().str.replace('-','').str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('"','').str.replace('.','')
    return df

### Original survey matrix

In [5]:
pricing_matrix = col_name(pd.read_excel(os.path.join(path,file_survey)))
pricing_matrix = pricing_matrix.replace({'0': np.nan, 0: np.nan})

In [6]:
col_scale = [x+'_scaled' for x in list(pricing_matrix.columns)[2:]]
col = list(pricing_matrix.columns) + col_scale

In [7]:
min_max_scaler = preprocessing.MinMaxScaler()
df_scale = pd.DataFrame(columns=col)
for u in pricing_matrix.eau.unique():
    df_eau = (pricing_matrix[pricing_matrix.eau == u]).reset_index(drop=True)
    x = df_eau.iloc[:,2:].values
    x_scaled = min_max_scaler.fit_transform(x)
    df_temp_1 = pd.DataFrame(x_scaled, columns=col_scale).reset_index(drop=True)
    df_temp_concat = pd.concat([df_eau,df_temp_1], ignore_index=True, axis=1)
    df_temp_concat.columns = col
    df_scale = pd.concat([df_scale, df_temp_concat], axis=0)    

In [8]:
df_scale[col_scale] = df_scale[col_scale].fillna(1)

### groupby EAU for scaler

In [9]:
complexity_level = {'high': ['high1_scaled', 'high2_scaled', 'high3_scaled', 'high4_scaled', 'high5_scaled', 'high_scaled'], 
                    'low': ['low1_scaled', 'low2_scaled', 'low3_scaled', 'low4_scaled', 'low5_scaled', 'low6_scaled', 'low_scaled'],
                    'med': ['med10_scaled', 'med1_scaled', 'med2_scaled', 'med3_scaled', 'med4_scaled', 'med5_scaled', 'med6_scaled', 'med7_scaled', 'med8_scaled', 'med9_scaled', 'med_scaled']}

In [10]:
df_scale

,supplier,eau,low,med,med1,med2,med3,med4,low1,low2,med5,med6,high,high1,high2,high3,med7,med8,low3,low4,high4,high5,med9,med10,low5,low6,low_scaled,med_scaled,med1_scaled,med2_scaled,med3_scaled,med4_scaled,low1_scaled,low2_scaled,med5_scaled,med6_scaled,high_scaled,high1_scaled,high2_scaled,high3_scaled,med7_scaled,med8_scaled,low3_scaled,low4_scaled,high4_scaled,high5_scaled,med9_scaled,med10_scaled,low5_scaled,low6_scaled
0,CVG,1,103.214105,248.982700,144.084700,426.629500,904.749100,240.709800,53.320000,44.029700,241.190000,231.085500,486.878600,634.290000,2348.090000,1594.319800,733.100000,355.479800,163.870130,60.564110,1061.538200,586.600000,99.835600,53.309300,65.460400,20.348200,0.129403,0.030610,0.071076,0.120174,0.109128,0.078925,0.038979,0.048006,0.114341,0.066630,0.088251,0.079307,0.270460,0.111282,0.046303,0.057422,0.085068,0.036203,0.027571,0.036198,0.007075,0.001912,0.000000,0.018222
1,ICA,1,92.433519,242.571989,62.766199,344.145584,616.139045,149.131019,42.859080,37.466633,118.171918,177.122042,300.719250,395.904943,1173.657405,1023.909023,537.500334,272.457737,135.663307,58.224159,1034.606316,640.766072,128.223468,81.081545,115.979536,28.564345,0.106486,0.024925,0.000000,0.072283,0.022344,0.000000,0.008505,0.032807,0.015737,0.031088,0.016721,0.013716,0.070043,0.018801,0.000000,0.019816,0.057673,0.030772,0.022405,0.047874,0.030574,0.038503,0.060535,0.047417
2,Julian,1,80.905360,214.463264,78.698537,420.672553,541.832685,212.488497,60.838274,31.499098,98.538938,219.605698,369.452909,346.057406,NaN,NaN,703.129572,347.545709,131.341119,60.384008,NaN,NaN,265.053149,61.722135,130.215281,29.230972,0.081980,0.000000,0.013926,0.116716,0.000000,0.054603,0.060881,0.018988,0.000000,0.059069,0.043131,0.000000,1.000000,1.000000,0.039208,0.053828,0.053476,0.035785,1.000000,1.000000,0.143839,0.012996,0.077594,0.049785
3,LMI,1,503.867431,1342.193194,1206.874681,1941.976201,3867.424826,1309.451515,383.211389,455.113389,1346.127083,1648.245828,2859.713681,3401.949411,6623.148549,7075.819125,4761.849083,2436.393183,1105.930871,475.814931,6131.400531,5057.601431,1299.333278,810.846556,707.786736,296.648042,0.981088,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.840829,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.769679,1.000000
4,MCL,1,199.097167,412.203983,195.549241,755.707442,1577.741929,329.072600,NaN,NaN,NaN,NaN,NaN,NaN,1794.177393,2217.330188,992.026047,557.787140,361.991472,103.447326,2724.078961,NaN,353.159392,209.962586,NaN,74.922349,0.333225,0.175344,0.116058,0.311240,0.311496,0.155079,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.175935,0.212291,0.107597,0.149060,0.277484,0.135735,0.346456,1.000000,0.216772,0.208309,1.000000,0.212141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,Wise Harness,5,112.983040,449.272058,162.657080,538.666184,1108.753143,287.725810,111.417520,58.586212,281.271617,314.250201,596.945659,666.435656,2199.474475,2215.991271,918.179798,410.786709,208.294022,72.926555,2359.489750,1212.231800,253.074080,137.632280,92.479850,42.126980,0.213344,0.341381,0.134367,0.241803,0.280596,0.191605,0.312133,0.121855,0.213623,0.186862,0.204082,0.180104,0.306926,0.302975,0.171291,0.139472,0.173480,0.129603,0.385319,0.225554,0.186102,0.168145,0.058428,0.137249
15,Dekko,5,81.720868,200.768552,NaN,276.028069,539.170939,167.474590,39.640360,22.970933,114.125885,144.440337,218.228071,371.715972,NaN,NaN,467.323493,210.324851,176.379680,40.830996,874.043690,457.884400,109.611080,70.702665,140.772300,22.841100,0.132030,0.091941,1.000000,0.054600,0.081088,0.069569,0.063705,0.023672,0.057770,0.048142,0.032525,0.065478,1.000000,1.000000,0.048254,0.030111,0.135764,0.040658,0.051306,0.026631,0.042133,0.061575,0.146816,0.053688
16,Kato,5,78.953760,195.950070,64.050000

In [11]:
for v in complexity_level.values(): 
    df_scale = pd.concat([df_scale, df_scale[v].mean(axis=1, skipna=True)], axis=1)

In [12]:
new_median = ['high_median', 'low_median', 'med_median']
new_cols = list(df_scale.columns)[:-3] + new_median
df_scale.columns = new_cols

In [13]:
df_scale[['high_price_rank', 'low_price_rank', 'med_price_rank']] = df_scale.groupby(['eau'])[['high_median', 'low_median', 'med_median']].rank(method = 'dense')

In [14]:
df_scale.to_csv(os.path.join(result_path, result_file), index=False)

### Current_State_Analysis

In [15]:
df = pd.read_excel(os.path.join(path,file_survey),sheet_name = 'current', header=None)

In [16]:
current_state = df.transpose().reset_index(drop=True)
current_state.columns = current_state.iloc[0]
current_state.drop(index = 0, inplace=True)
current_state = col_name(current_state.reset_index(drop=True))
current_state['current_price'] = current_state.current_price.astype('float')

In [17]:
df_eau = col_name(pd.read_excel(os.path.join(path,file_survey),sheet_name = 'EAU'))
df_eau.segment.replace('FIRE AND EMERGENCY', 'F&E', inplace = True)
df_eau.drop_duplicates(subset='segment', inplace=True)
df_eau.set_index('segment', inplace=True)

In [18]:
def allocate_eau(df_eau, x):
    if x[0] < df_eau.loc[x[1], 'eau2']:
        return 1
    elif x[0] < df_eau.loc[x[1], 'eau3']:
        return 2
    elif x[0] < df_eau.loc[x[1], 'eau4']:
        return 3
    elif x[0] < df_eau.loc[x[1], 'eau4']:
        return 4
    else:
        return 5

In [19]:
current_state['eau_group'] = current_state[['current_eau','segment']].apply(lambda x: allocate_eau(df_eau, x),axis=1)

In [20]:
df_scale

,supplier,eau,low,med,med1,med2,med3,med4,low1,low2,med5,med6,high,high1,high2,high3,med7,med8,low3,low4,high4,high5,med9,med10,low5,low6,low_scaled,med_scaled,med1_scaled,med2_scaled,med3_scaled,med4_scaled,low1_scaled,low2_scaled,med5_scaled,med6_scaled,high_scaled,high1_scaled,high2_scaled,high3_scaled,med7_scaled,med8_scaled,low3_scaled,low4_scaled,high4_scaled,high5_scaled,med9_scaled,med10_scaled,low5_scaled,low6_scaled,high_median,low_median,med_median,high_price_rank,low_price_rank,med_price_rank
0,CVG,1,103.214105,248.982700,144.084700,426.629500,904.749100,240.709800,53.320000,44.029700,241.190000,231.085500,486.878600,634.290000,2348.090000,1594.319800,733.100000,355.479800,163.870130,60.564110,1061.538200,586.600000,99.835600,53.309300,65.460400,20.348200,0.129403,0.030610,0.071076,0.120174,0.109128,0.078925,0.038979,0.048006,0.114341,0.066630,0.088251,0.079307,0.270460,0.111282,0.046303,0.057422,0.085068,0.036203,0.027571,0.036198,0.007075,0.001912,0.000000,0.018222,0.102178,0.050840,0.063963,6.0,2.0,4.0
1,ICA,1,92.433519,242.571989,62.766199,344.145584,616.139045,149.131019,42.859080,37.466633,118.171918,177.122042,300.719250,395.904943,1173.657405,1023.909023,537.500334,272.457737,135.663307,58.224159,1034.606316,640.766072,128.223468,81.081545,115.979536,28.564345,0.106486,0.024925,0.000000,0.072283,0.022344,0.000000,0.008505,0.032807,0.015737,0.031088,0.016721,0.013716,0.070043,0.018801,0.000000,0.019816,0.057673,0.030772,0.022405,0.047874,0.030574,0.038503,0.060535,0.047417,0.031593,0.049171,0.023206,1.0,1.0,1.0
2,Julian,1,80.905360,214.463264,78.698537,420.672553,541.832685,212.488497,60.838274,31.499098,98.538938,219.605698,369.452909,346.057406,NaN,NaN,703.129572,347.545709,131.341119,60.384008,NaN,NaN,265.053149,61.722135,130.215281,29.230972,0.081980,0.000000,0.013926,0.116716,0.000000,0.054603,0.060881,0.018988,0.000000,0.059069,0.043131,0.000000,1.000000,1.000000,0.039208,0.053828,0.053476,0.035785,1.000000,1.000000,0.143839,0.012996,0.077594,0.049785,0.673855,0.054070,0.044926,15.0,4.0,3.0
3,LMI,1,503.867431,1342.193194,1206.874681,1941.976201,3867.424826,1309.451515,383.211389,455.113389,1346.127083,1648.245828,2859.713681,3401.949411,6623.148549,7075.819125,4761.849083,2436.393183,1105.930871,475.814931,6131.400531,5057.601431,1299.333278,810.846556,707.786736,296.648042,0.981088,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.840829,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.769679,1.000000,0.973471,0.964395,1.000000,18.0,19.0,19.0
4,MCL,1,199.097167,412.203983,195.549241,755.707442,1577.741929,329.072600,NaN,NaN,NaN,NaN,NaN,NaN,1794.177393,2217.330188,992.026047,557.787140,361.991472,103.447326,2724.078961,NaN,353.159392,209.962586,NaN,74.922349,0.333225,0.175344,0.116058,0.311240,0.311496,0.155079,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.175935,0.212291,0.107597,0.149060,0.277484,0.135735,0.346456,1.000000,0.216772,0.208309,1.000000,0.212141,0.622447,0.565512,0.340996,14.0,17.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,Wise Harness,5,112.983040,449.272058,162.657080,538.666184,1108.753143,287.725810,111.417520,58.586212,281.271617,314.250201,596.945659,666.435656,2199.474475,2215.991271,918.179798,410.786709,208.294022,72.926555,2359.489750,1212.231800,253.074080,137.632280,92.479850,42.126980,0.213344,0.341381,0.134367,0.241803,0.280596,0.191605,0.312133,0.121855,0.213623,0.186862,0.204082,0.180104,0.306926,0.302975,0.171291,0.139472,0.173480,0.129603,0.385319,0.225554,0.186102,0.168145,0.058428,0.137249,0.267493,0.163727,0.205022,12.0,15.0,14.0
15,Dekko,5,81.720868,200.768552,NaN,276.028069,539.170939,167.474590,39.640360,22.970933,114.125885,144.440337,218.228071,371.715972,NaN,NaN,467.323493,210.

In [21]:
current_state['complexity_level'] = df_scale.columns[2: 26]

In [22]:
current_state

,part#,current_supplier,current_price,current_eau,ext_spend,segment,supplier,eau_group,complexity_level
0,1596988,Unlimited,58.460,500,29230,COMMERCIAL,Low,5,low
1,1568934,KATO,205.950,100,20595,COMMERCIAL,Med,2,med
2,77441612,Unlimited,37.630,1670,62842.1,COMMERCIAL,Med,5,med1
3,1614820,ECI,346.460,100,34646,COMMERCIAL,Med,2,med2
4,1564491,Unlimited,371.120,480,178138,COMMERCIAL,Med,3,med3
5,77441537,Unlimited,106.780,817,87239.3,COMMERCIAL,Med,5,med4
6,1001194114,ECI,28.180,5,140.9,ACCESS,Low,1,low1
7,1001169151,ECI,33.240,1765,58668.6,ACCESS,Low,5,low2
8,1001238407,ECI,62.350,1716,106993,ACCESS,Med,5,med5
9,1001168952,ECI,90.480,1388,125586,ACCESS,Med,3,med6


In [23]:
conditions = [
    (current_state.supplier == 'Low'),
    (current_state.supplier == 'Med'),
    (current_state.supplier == 'High')
]

values = ['low_price_rank', 'med_price_rank', 'high_price_rank']

current_state['complexity_rank'] = np.select(conditions, values)

In [24]:
# test = df_scale[df_scale.eau == current_state.eau_group[0]][['supplier' ,current_state.complexity_level[0], current_state.complexity_rank[0]]].sort_values(by = current_state.complexity_rank[0])
# test = df_scale[df_scale.eau == current_state.eau_group[0]][[current_state.complexity_level[0], current_state.complexity_rank[0]]].sort_values(by = current_state.complexity_rank[0])

In [25]:
rank_price_col = ['rank_price_'+str(x) for x in range(1,20)]

In [26]:
def rank_price(df_scale, x):
    df = df_scale[df_scale.eau == x[0]]
    df = df[[x[1], x[2]]].sort_values(by = x[2])
    res = list(df.iloc[:,0])
    return res 

In [27]:
prices = current_state[['eau_group','complexity_level', 'complexity_rank']].apply(lambda x: rank_price(df_scale, x), axis=1)

In [28]:
df_pricing_rank = pd.DataFrame(columns = rank_price_col)

In [29]:
for i in range(len(prices)):
    item = pd.DataFrame([prices[i]], columns= rank_price_col)
    df_pricing_rank = pd.concat([df_pricing_rank, item]).reset_index(drop = True)

In [30]:
current_state[rank_price_col] = df_pricing_rank

In [31]:
current_state

,part#,current_supplier,current_price,current_eau,ext_spend,segment,supplier,eau_group,complexity_level,complexity_rank,rank_price_1,rank_price_2,rank_price_3,rank_price_4,rank_price_5,rank_price_6,rank_price_7,rank_price_8,rank_price_9,rank_price_10,rank_price_11,rank_price_12,rank_price_13,rank_price_14,rank_price_15,rank_price_16,rank_price_17,rank_price_18,rank_price_19
0,1596988,Unlimited,58.460,500,29230,COMMERCIAL,Low,5,low,low_price_rank,43.684500,47.198313,57.181277,72.342428,45.927294,55.990476,81.516100,81.231212,81.720868,82.990000,78.953760,97.051160,30.960000,111.640000,112.983040,76.927043,139.290069,90.536463,415.423542
1,1568934,KATO,205.950,100,20595,COMMERCIAL,Med,2,med,med_price_rank,147.261608,134.699600,211.090538,160.514811,175.321500,198.559512,379.112746,259.110000,246.022796,235.950070,334.790000,237.780552,264.139504,450.322058,346.669357,311.239662,337.646017,NaN,1342.193194
2,77441612,Unlimited,37.630,1670,62842.1,COMMERCIAL,Med,5,med1,med_price_rank,44.255032,38.335553,45.594700,40.557716,34.901792,39.362221,66.152400,61.120000,64.050000,45.437121,92.780000,NaN,NaN,162.657080,128.962358,77.538300,63.846872,NaN,985.694125
3,1614820,ECI,346.460,100,34646,COMMERCIAL,Med,2,med2,med_price_rank,230.886477,252.558100,307.768601,300.991106,277.311400,325.170760,277.473437,393.560000,305.027413,315.950000,447.060000,327.418069,356.218558,539.176184,615.253247,417.359300,NaN,212.200000,1940.601201
4,1564491,Unlimited,371.120,480,178138,COMMERCIAL,Med,3,med3,med_price_rank,315.555786,342.040042,431.929000,387.963287,437.872738,659.311386,540.532500,490.550000,589.950000,662.959327,815.780000,551.318535,580.984939,1109.223618,1232.357621,643.875500,NaN,NaN,3327.403090
5,77441537,Unlimited,106.780,817,87239.3,COMMERCIAL,Med,5,med4,med_price_rank,98.922830,108.064569,114.486800,130.459829,130.024126,159.333176,163.099900,134.910000,175.950000,144.059730,208.210000,113.355866,167.474590,287.725810,230.583892,165.140400,261.995583,NaN,1084.299570
6,1001194114,ECI,28.180,5,140.9,ACCESS,Low,1,low1,low_price_rank,42.859080,53.320000,67.190000,60.838274,39.939431,41.160000,119.503895,86.960114,NaN,73.479866,149.984689,45.856975,85.755900,99.950000,152.748070,150.000360,NaN,NaN,383.211389
7,1001169151,ECI,33.240,1765,58668.6,ACCESS,Low,5,low2,low_price_rank,14.851800,14.384306,19.356570,20.280912,18.442440,20.542475,21.250300,14.951581,22.970933,40.610000,30.950000,24.103226,15.410000,29.320000,58.586212,29.497408,NaN,NaN,377.124917
8,1001238407,ECI,62.350,1716,106993,ACCESS,Med,5,med5,med_price_rank,63.103191,74.928880,108.402700,52.169955,65.505470,60.077414,75.749600,85.590000,129.950000,91.630414,176.270000,77.918228,114.125885,281.271617,NaN,NaN,NaN,NaN,1124.628992
9,1001168952,ECI,90.480,1388,125586,ACCESS,Med,3,med6,med_price_rank,102.624180,132.739970,116.565500,104.381708,134.057951,90.505441,123.362100,140.930000,179.950000,195.630909,211.320000,130.965836,167.282337,314.379981,NaN,NaN,NaN,107.820000,1374.741870


In [32]:
def weighted_avg(x, w):
    if w == w5:
        return np.average([x[1], x[2], x[3], x[4], x[5]], weights = w)
    elif w == w3:
        return np.average([x[1], x[2], x[3]], weights = w)

In [33]:
w5 = [0.4, 0.3, 0.1, 0.1, 0.1]
w3 = [0.6, 0.3, 0.1]
current_state['rank3_avg_price'] = current_state[['current_supplier','rank_price_1', 'rank_price_2', 'rank_price_3']].apply(lambda x: weighted_avg(x, w3), axis=1)
current_state['rank5_avg_price'] = current_state[['current_supplier','rank_price_1', 'rank_price_2', 'rank_price_3','rank_price_4', 'rank_price_5']].apply(lambda x: weighted_avg(x, w5), axis=1)
current_state['rank1_lowest_price'] = current_state[['rank_price_1', 'rank_price_2', 'rank_price_3','rank_price_4', 'rank_price_5']].apply(lambda x: min(x[0],x[1],x[2],x[3],x[4]), axis=1)


current_state['curr_higher_rank5_pct'] = current_state.rank5_avg_price/current_state.current_price
current_state['curr_higher_rank3_pct'] = current_state.rank3_avg_price/current_state.current_price
current_state['curr_higher_lowestP_pct'] = current_state.rank1_lowest_price/current_state.current_price

In [34]:
current_state[['part#','current_price', 'current_supplier','supplier', 'eau_group','rank1_lowest_price', 'curr_higher_lowestP_pct', 
               'rank3_avg_price', 'curr_higher_rank3_pct', 'rank5_avg_price','curr_higher_rank5_pct']].sort_values(by= ['supplier', 'eau_group'])

,part#,current_price,current_supplier,supplier,eau_group,rank1_lowest_price,curr_higher_lowestP_pct,rank3_avg_price,curr_higher_rank3_pct,rank5_avg_price,curr_higher_rank5_pct
10,1001163370,127.770,ECI,High,1,300.719250,2.353598,350.497503,2.743191,379.413170,2.969501
12,64-3600-H003,2641.140,ICA,High,2,754.670000,0.285736,1053.128014,0.398740,1027.713222,0.389117
18,A000P725,710.000,ICA,High,3,709.997449,0.999996,754.635166,1.062866,784.177536,1.104475
19,A000P906,417.260,St Clair,High,3,382.468178,0.916618,405.428438,0.971645,415.667303,0.996183
11,1001238405,304.780,ECI,High,5,233.335841,0.765588,258.922267,0.849538,267.277479,0.876952
13,64-4851-H015,716.850,ECI,High,5,609.309097,0.849981,698.203672,0.973989,767.053380,1.070033
6,1001194114,28.180,ECI,Low,1,39.939431,1.417297,48.430448,1.718611,49.936403,1.772051
23,A000D729,126.510,UNLIMITED SERVICES,Low,1,20.348200,0.160843,25.785067,0.203818,25.907438,0.204786
22,A000D731,75.952,St Clair,Low,2,64.192100,0.845167,81.826908,1.077350,87.256818,1.148842
0,1596988,58.460,Unlimited,Low,5,43.684500,0.747255,46.088322,0.788374,49.178394,0.841232


In [35]:
current_state_group = current_state.groupby(['supplier','eau_group'])[['curr_higher_lowestP_pct', 'curr_higher_rank3_pct','curr_higher_rank5_pct']].mean().reset_index()

In [36]:
current_state_group

,supplier,eau_group,curr_higher_lowestP_pct,curr_higher_rank3_pct,curr_higher_rank5_pct
0,High,1,2.353598,2.743191,2.969501
1,High,2,0.285736,0.398740,0.389117
2,High,3,0.958307,1.017256,1.050329
3,High,5,0.807785,0.911763,0.973493
4,Low,1,0.789070,0.961215,0.988418
5,Low,2,0.845167,1.077350,1.148842
6,Low,5,0.573644,0.613399,0.670458
7,Med,2,0.974704,1.047286,1.092914
8,Med,3,0.842777,1.052748,1.081783
9,Med,5,0.896881,1.080245,1.083678


### Financial Impact

In [37]:
competitive_matrix_raw = col_name(pd.read_excel(os.path.join(path,file_full)))
complexity_list = ['1 Low 2-20 circuits', '2 Medium 20-100 circuits', '3 High 100+ circuits']
competitive_matrix = competitive_matrix_raw[competitive_matrix_raw.complexity.isin(complexity_list)].drop(columns=['eau1', 'eau2','eau3','eau4','eau5'])

In [38]:
conditions2 = [
    (competitive_matrix.complexity == '1 Low 2-20 circuits'),
    (competitive_matrix.complexity == '2 Medium 20-100 circuits'),
    (competitive_matrix.complexity == '3 High 100+ circuits')
]

values = ['Low', 'Med', 'High']

competitive_matrix['complexity_rank'] = np.select(conditions2, values)

In [40]:
competitive_matrix.head()

,parent_supplier_name,part_number,part_description,invoice_qty,eau_15,avg_unit_cost,total_spend_invoiced,segment,facility_division,gpsc_commodity_classification,subcommodity,complexity,eau,complexity_rank
1225,"L3HARRIS TECHNOLOGIES, INC.",12604466,"WIREHARN,COMPT,GFESIGW189",1985,NaN,1945.11,3861043.35,DEFENSE,DEFENSE PRODUCTION,ELECTRICAL,WIREHARNESS,1 Low 2-20 circuits,4,Low
1226,MIDWEST HARNESS & CABLE (FORMERLY DDI),12602284,"WIREHARN,CAB,CREWAFES",3891,NaN,303.65,1172290.53,DEFENSE,DEFENSE PRODUCTION,ELECTRICAL,WIREHARNESS,1 Low 2-20 circuits,4,Low
1227,ST CLAIR TECHNOLOGIES,12604471,"WIREHARN,CAB,GFEPWRP104",2276,NaN,278.67,634252.36,DEFENSE,DEFENSE PRODUCTION,ELECTRICAL,WIREHARNESS,1 Low 2-20 circuits,4,Low
1228,ST CLAIR TECHNOLOGIES,12603056,"WIREHARN,CAB,GFEPWRP105",2253,NaN,229.02,516742.30,DEFENSE,DEFENSE PRODUCTION,ELECTRICAL,WIREHARNESS,1 Low 2-20 circuits,4,Low
1229,FALMAT INC.,12603049,"WIREHARN,COMPT,GICW131",2175,NaN,222.20,483285.00,DEFENSE,DEFENSE PRODUCTION,ELECTRICAL,WIREHARNESS,1 Low 2-20 circuits,4,Low


In [41]:
competitive_financial_result = pd.merge(competitive_matrix, current_state_group, how = 'left', left_on=['complexity_rank', 'eau'], right_on=['supplier', 'eau_group']).drop(columns = ['complexity_rank', 'supplier', 'eau_group'])

In [42]:
competitive_financial_result.to_csv(os.path.join(result_path, result_financial_impact), index=False)